In [1]:
import tensorflow as tf
from pathlib import Path
from util.dataset import Dataset as madras_Dataset
from util import read_data as rd
from tensorflow.keras import layers
from madras_laftr import trainer, tester
from madras_laftr.models import *
from util.results import ResultLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.data import Dataset
from util import metrics
import numpy as np

In [2]:
batch_size = 64
epochs = 1000
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)
CLASS_COEFF = 1.
FAIR_COEFF = 1.
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [3]:
data_name = 'adult'

data_info = rd.return_data_info(data_name)
npzfile = rd.return_npz(data_name)

data = madras_Dataset(npzfile=npzfile, name=data_name, a0_name=data_info['a0_name'], a1_name=data_info['a1_name'], 
                use_a=data_info['use_a'], seed=data_info['seed'], batch_size=batch_size)

data_shapes = list(data.get_shapes())
#print(data_shapes)

xdim = data_shapes[0][1]
ydim = data_shapes[1][1]
adim = data_shapes[2][1]
zdim = 8

In [4]:
train_data = Dataset.from_tensor_slices((data.x_train, data.y_train, data.a_train))
train_data = train_data.batch(batch_size)
train_data

<BatchDataset shapes: ((None, 113), (None, 1), (None, 1)), types: (tf.float64, tf.float32, tf.float32)>

In [5]:
valid_data = Dataset.from_tensor_slices((data.x_valid, data.y_valid, data.a_valid))
valid_data = valid_data.batch(batch_size)

In [6]:
def train_unfair(unfair_clas, X, Y, optimizer, learning_rate=0.001):
    
    with tf.GradientTape() as tape0:
        
        unfair_clas(X, Y) #to compute the foward
        current_loss = unfair_clas.loss #current loss
        #print('pre enc-clas-dec {}'.format(loss2min))
    
    grads = tape0.gradient(current_loss, unfair_clas.variables)
    #print(grads[-1])
    optimizer.apply_gradients(zip(grads, unfair_clas.variables))

In [7]:
def train_unfair_loop(unfair_clas, train_dataset, epochs, optmizer):
    
    print("> Epoch | Class Loss | Class Acc")

    losses = []
    accs = []
    dps = []
    dis = []
    
    for epoch in range(0, epochs):
        epoch_acc = []
        dp = []
        di = []
        
        for X, Y, A in train_dataset:
            
            train_unfair(unfair_clas, X, Y, optmizer)

            epoch_acc.append(metrics.accuracy(Y, tf.math.round(unfair_clas.Y_hat)))

            dp.append(metrics.DP(tf.math.round(unfair_clas.Y_hat).numpy(), A))
            di.append(metrics.DI(Y.numpy(), tf.math.round(unfair_clas.Y_hat).numpy(), A))

        losses.append(unfair_clas.loss)
        accs.append(tf.math.reduce_mean(epoch_acc))

        dps.append(tf.math.reduce_mean(dp))
        dis.append(tf.math.reduce_mean(di))
    
        print("> {} | {} | {}".format(
            epoch+1, 
            tf.math.reduce_mean(losses[-1]),
            accs[-1]))

    return tf.math.reduce_mean(losses[-1]), accs[-1], dps, dis

In [14]:
def validation(model, valid_data):
    losses = {'clas':[], 'dec':[], 'adv':[], 'model':[]}
    accs = {'clas':[], 'dec':[], 'adv':[]}
    
    for X, Y, A in valid_data:
        
        model(X, Y)
        
        losses['model'].append(model.loss)
        accs['clas'].append(metrics.accuracy(Y, tf.math.round(model.Y_hat)))
    
    print("> Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc")
    print("> {} | {} | {} | {} | {} | {} | {}".format(
            tf.math.reduce_mean(losses['model'][-1]),
            tf.math.reduce_mean(losses['clas']),
            tf.math.reduce_mean(losses['adv']), 
            tf.math.reduce_mean(losses['dec']), 
            tf.math.reduce_mean(accs['clas']),
            tf.math.reduce_mean(accs['adv']), 
            tf.math.reduce_mean(accs['dec'])))

    return losses, accs

In [9]:
unfairmlp = UnfairMLP(xdim, zdim, ydim)

In [15]:
train_unfair_loop(unfairmlp, train_data, 1, opt)

> Epoch | Class Loss | Class Acc
/home/luiz/ufpb/mestrado/code/falsb/util/metrics.py:79: RuntimeWarning: invalid value encountered in float_scalars
  return FN(Y, Ypred) / pos(Y)
> 1 | 1.3505558967590332 | 0.789290450928382


(<tf.Tensor: shape=(), dtype=float32, numpy=1.3505559>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.789290450928382>,
 [<tf.Tensor: shape=(), dtype=float32, numpy=0.13864583>],
 [<tf.Tensor: shape=(), dtype=float64, numpy=nan>])

In [16]:
loss, acc = validation(unfairmlp, valid_data)

> Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 0.8018909692764282 | nan | nan | nan | 0.789561170212766 | nan | nan
